Goal: convert a nuscenes point cloud to a .las file for visualization.

In [41]:
import os
from pathlib import Path
import numpy as np
import argparse
import tqdm
import pickle
from functools import reduce
from pyquaternion import Quaternion
from nuscenes.nuscenes import NuScenes
from nuscenes.utils import splits
from nuscenes.utils.geometry_utils import transform_matrix
from typing import Optional, Union
import laspy
import numpy as np
import pyproj
from dataclasses import dataclass

from pointcept.datasets.preprocessing.nuscenes.preprocess_nuscenes_info import \
    get_available_scenes, get_sample_data, obtain_sensor2top, \
    quaternion_yaw, map_name_from_general_to_detection
from pointcept.datasets.nuscenes import NuScenesDataset

In [2]:
dataset_root='/mnt/d/nuscenes/nuScenes-lidarseg-all-v1.0'
nusc_trainval = NuScenes(
    version="v1.0-trainval", dataroot=dataset_root, verbose=False
)

In [17]:
available_scenes_trainval = get_available_scenes(nusc_trainval)
available_scene_names_trainval = [s["name"] for s in available_scenes_trainval]
len(available_scenes_trainval)

850

In [18]:
train_scenes = set(
    [
        available_scenes_trainval[available_scene_names_trainval.index(s)]["token"]
        for s in splits.train[:1] # 1st one only
    ]
)
train_scenes

{'73030fb67d3c46cfb5e590168088ae39'}

In [16]:
# scene=nusc_trainval.scene[0]
# scene_token = scene["token"]
# scene_rec = nusc_trainval.get("scene", scene_token)
# sample_rec = nusc_trainval.get("sample", scene_rec["first_sample_token"])
# sd_rec = nusc_trainval.get("sample_data", sample_rec["data"]["LIDAR_TOP"])
# lidar_path, boxes, _ = nusc_trainval.get_sample_data(sd_rec["token"])
# Path(lidar_path).exists()

True

In [26]:
def fill_trainval_infos(
    data_path, nusc, train_scenes, test=False, max_sweeps=10, with_camera=False,
    train_only=False
):
    train_nusc_infos = []
    val_nusc_infos = []

    ref_chan = "LIDAR_TOP"  # The radar channel from which we track back n sweeps to aggregate the point cloud.
    chan = "LIDAR_TOP"  # The reference channel of the current sample_rec that the point clouds are mapped to.

    samples=list(filter(lambda x: x["scene_token"] in train_scenes, nusc.sample)) if train_only else nusc.sample
    progress_bar = tqdm.tqdm(
        total=len(samples), desc="create_info", dynamic_ncols=True
    )
    for index, sample in enumerate(samples):
        progress_bar.update()

        ref_sd_token = sample["data"][ref_chan]
        ref_sd_rec = nusc.get("sample_data", ref_sd_token)
        ref_cs_rec = nusc.get(
            "calibrated_sensor", ref_sd_rec["calibrated_sensor_token"]
        )
        ref_pose_rec = nusc.get("ego_pose", ref_sd_rec["ego_pose_token"])
        ref_time = 1e-6 * ref_sd_rec["timestamp"]

        ref_lidar_path, ref_boxes, _ = get_sample_data(nusc, ref_sd_token)

        ref_cam_front_token = sample["data"]["CAM_FRONT"]
        ref_cam_path, _, ref_cam_intrinsic = nusc.get_sample_data(ref_cam_front_token)

        # Homogeneous transform from ego car frame to reference frame
        ref_from_car = transform_matrix(
            ref_cs_rec["translation"], Quaternion(ref_cs_rec["rotation"]), inverse=True
        )

        # Homogeneous transformation matrix from global to _current_ ego car frame
        car_from_global = transform_matrix(
            ref_pose_rec["translation"],
            Quaternion(ref_pose_rec["rotation"]),
            inverse=True,
        )
        info = {
            "lidar_path": Path(ref_lidar_path).relative_to(data_path).__str__(),
            "lidar_token": ref_sd_token,
            "cam_front_path": Path(ref_cam_path).relative_to(data_path).__str__(),
            "cam_intrinsic": ref_cam_intrinsic,
            "token": sample["token"],
            "sweeps": [],
            "ref_from_car": ref_from_car,
            "car_from_global": car_from_global,
            "timestamp": ref_time,
        }
        if with_camera:
            info["cams"] = dict()
            l2e_r = ref_cs_rec["rotation"]
            l2e_t = (ref_cs_rec["translation"],)
            e2g_r = ref_pose_rec["rotation"]
            e2g_t = ref_pose_rec["translation"]
            l2e_r_mat = Quaternion(l2e_r).rotation_matrix
            e2g_r_mat = Quaternion(e2g_r).rotation_matrix

            # obtain 6 image's information per frame
            camera_types = [
                "CAM_FRONT",
                "CAM_FRONT_RIGHT",
                "CAM_FRONT_LEFT",
                "CAM_BACK",
                "CAM_BACK_LEFT",
                "CAM_BACK_RIGHT",
            ]
            for cam in camera_types:
                cam_token = sample["data"][cam]
                cam_path, _, camera_intrinsics = nusc.get_sample_data(cam_token)
                cam_info = obtain_sensor2top(
                    nusc, cam_token, l2e_t, l2e_r_mat, e2g_t, e2g_r_mat, cam
                )
                cam_info["data_path"] = (
                    Path(cam_info["data_path"]).relative_to(data_path).__str__()
                )
                cam_info.update(camera_intrinsics=camera_intrinsics)
                info["cams"].update({cam: cam_info})

        sample_data_token = sample["data"][chan]
        curr_sd_rec = nusc.get("sample_data", sample_data_token)
        sweeps = []
        while len(sweeps) < max_sweeps - 1:
            if curr_sd_rec["prev"] == "":
                if len(sweeps) == 0:
                    sweep = {
                        "lidar_path": Path(ref_lidar_path)
                        .relative_to(data_path)
                        .__str__(),
                        "sample_data_token": curr_sd_rec["token"],
                        "transform_matrix": None,
                        "time_lag": curr_sd_rec["timestamp"] * 0,
                    }
                    sweeps.append(sweep)
                else:
                    sweeps.append(sweeps[-1])
            else:
                curr_sd_rec = nusc.get("sample_data", curr_sd_rec["prev"])

                # Get past pose
                current_pose_rec = nusc.get("ego_pose", curr_sd_rec["ego_pose_token"])
                global_from_car = transform_matrix(
                    current_pose_rec["translation"],
                    Quaternion(current_pose_rec["rotation"]),
                    inverse=False,
                )

                # Homogeneous transformation matrix from sensor coordinate frame to ego car frame.
                current_cs_rec = nusc.get(
                    "calibrated_sensor", curr_sd_rec["calibrated_sensor_token"]
                )
                car_from_current = transform_matrix(
                    current_cs_rec["translation"],
                    Quaternion(current_cs_rec["rotation"]),
                    inverse=False,
                )

                tm = reduce(
                    np.dot,
                    [ref_from_car, car_from_global, global_from_car, car_from_current],
                )

                lidar_path = nusc.get_sample_data_path(curr_sd_rec["token"])

                time_lag = ref_time - 1e-6 * curr_sd_rec["timestamp"]

                sweep = {
                    "lidar_path": Path(lidar_path).relative_to(data_path).__str__(),
                    "sample_data_token": curr_sd_rec["token"],
                    "transform_matrix": tm,
                    "global_from_car": global_from_car,
                    "car_from_current": car_from_current,
                    "time_lag": time_lag,
                }
                sweeps.append(sweep)

        info["sweeps"] = sweeps

        assert len(info["sweeps"]) == max_sweeps - 1, (
            f"sweep {curr_sd_rec['token']} only has {len(info['sweeps'])} sweeps, "
            f"you should duplicate to sweep num {max_sweeps - 1}"
        )

        if not test:
            # processing gt bbox
            annotations = [
                nusc.get("sample_annotation", token) for token in sample["anns"]
            ]

            # the filtering gives 0.5~1 map improvement
            num_lidar_pts = np.array([anno["num_lidar_pts"] for anno in annotations])
            num_radar_pts = np.array([anno["num_radar_pts"] for anno in annotations])
            mask = num_lidar_pts + num_radar_pts > 0

            locs = np.array([b.center for b in ref_boxes]).reshape(-1, 3)
            dims = np.array([b.wlh for b in ref_boxes]).reshape(-1, 3)[
                :, [1, 0, 2]
            ]  # wlh == > dxdydz (lwh)
            velocity = np.array([b.velocity for b in ref_boxes]).reshape(-1, 3)
            rots = np.array([quaternion_yaw(b.orientation) for b in ref_boxes]).reshape(
                -1, 1
            )
            names = np.array([b.name for b in ref_boxes])
            tokens = np.array([b.token for b in ref_boxes])
            gt_boxes = np.concatenate([locs, dims, rots, velocity[:, :2]], axis=1)

            assert len(annotations) == len(gt_boxes) == len(velocity)

            info["gt_boxes"] = gt_boxes[mask, :]
            info["gt_boxes_velocity"] = velocity[mask, :]
            info["gt_names"] = np.array(
                [map_name_from_general_to_detection[name] for name in names]
            )[mask]
            info["gt_boxes_token"] = tokens[mask]
            info["num_lidar_pts"] = num_lidar_pts[mask]
            info["num_radar_pts"] = num_radar_pts[mask]

            # processing gt segment
            segment_path = nusc.get("lidarseg", ref_sd_token)["filename"]
            info["gt_segment_path"] = segment_path

        if sample["scene_token"] in train_scenes:
            train_nusc_infos.append(info)
        else:
            val_nusc_infos.append(info)

    progress_bar.close()
    return train_nusc_infos, val_nusc_infos


In [27]:
max_sweeps=10
with_camera=False
train_nusc_infos, _ = fill_trainval_infos(
    dataset_root,
    nusc_trainval,
    train_scenes,
    test=False,
    max_sweeps=max_sweeps,
    with_camera=with_camera,
    train_only=True
)



create_info: 100%|██████████| 40/40 [00:00<00:00, 192.20it/s]


In [29]:
output_root='../data'
os.makedirs(os.path.join(output_root, "info"), exist_ok=True)
with open(
    os.path.join(
        output_root,
        "info",
        f"nuscenes_infos_{max_sweeps}sweeps_train.pkl",
    ),
    "wb",
) as f:
    pickle.dump(train_nusc_infos, f)

In [37]:
# type:ignore
train_ds=NuScenesDataset(**dict(
    # type='NuScenesDataset',
    split="train",
    data_root=dataset_root,
    transform=[
        # dict(type="RandomDropout", dropout_ratio=0.2, dropout_application_ratio=0.2),
        # dict(type="RandomRotateTargetAngle", angle=(1/2, 1, 3/2), center=[0, 0, 0], axis="z", p=0.75),
        dict(type="RandomRotate", angle=[-1, 1], axis="z", center=[0, 0, 0], p=0.5),
        # dict(type="RandomRotate", angle=[-1/6, 1/6], axis="x", p=0.5),
        # dict(type="RandomRotate", angle=[-1/6, 1/6], axis="y", p=0.5),
        dict(type="RandomScale", scale=[0.9, 1.1]),
        # dict(type="RandomShift", shift=[0.2, 0.2, 0.2]),
        dict(type="RandomFlip", p=0.5),
        dict(type="RandomJitter", sigma=0.005, clip=0.02),
        # dict(type="ElasticDistortion", distortion_params=[[0.2, 0.4], [0.8, 1.6]]),
        dict(
            type="GridSample",
            grid_size=0.05,
            hash_type="fnv",
            mode="train",
            keys=("coord", "strength", "segment"),
            return_grid_coord=True,
        ),
        # dict(type="SphereCrop", point_max=1000000, mode="random"),
        # dict(type="CenterShift", apply_z=False),
        dict(type="ToTensor"),
        dict(
            type="Collect",
            keys=("coord", "grid_coord", "segment"),
            feat_keys=("coord", "strength"),
        ),
    ],
    test_mode=False,
    ignore_index=-1,
))

[2025-02-26 22:31:52,313 INFO defaults.py line 68 75257] Totally 40 x 1 samples in train set.


In [43]:
sample=train_ds[0]
sample

{'coord': tensor([[  5.5512, -36.6194,  -1.7120],
         [  5.5730, -26.7211,   2.5473],
         [  5.4475, -27.8848,   4.6742],
         ...,
         [  8.8255, -20.9018,   4.2864],
         [  8.9210, -21.9589,  -1.6491],
         [  8.8434, -22.6365,   1.6966]]),
 'grid_coord': tensor([[2015, 1184,  199],
         [2015, 1382,  284],
         [2012, 1359,  327],
         ...,
         [2080, 1498,  319],
         [2082, 1477,  201],
         [2080, 1464,  267]]),
 'segment': tensor([13, 15, 15,  ..., 15, 12, 15]),
 'offset': tensor([22100]),
 'feat': tensor([[  5.5512, -36.6194,  -1.7120,   0.0588],
         [  5.5730, -26.7211,   2.5473,   0.0471],
         [  5.4475, -27.8848,   4.6742,   0.0627],
         ...,
         [  8.8255, -20.9018,   4.2864,   0.0510],
         [  8.9210, -21.9589,  -1.6491,   0.0392],
         [  8.8434, -22.6365,   1.6966,   0.0627]])}

In [54]:
@dataclass
class PointCloud:
    xyz: np.ndarray
    rgb: Optional[np.ndarray]
    crs: pyproj.CRS
    classification: Optional[np.ndarray]

class InvalidPointCloudError(Exception):
    pass

def check_point_cloud_validity(pcd:PointCloud):
    n_xyz=len(pcd.xyz)
    if not pcd.xyz.flags.contiguous:
        raise InvalidPointCloudError('xyz not contiguous')
    if pcd.rgb:
        n_rgb=len(pcd.rgb)
        if n_xyz!=n_rgb:
            raise InvalidPointCloudError(f'Length of coords ({n_xyz}) and colors ({n_rgb}) do not match')
        if not pcd.rgb.flags.contiguous:
            raise InvalidPointCloudError('rgb not contiguous')

def write_las_file(pcd: PointCloud, dst: Union[str,Path], replace:bool=False, mkparent:bool=True):
    check_point_cloud_validity(pcd)
    dst=Path(dst)
    if dst.exists() and not replace:
        raise FileExistsError(f'{dst} already exists, set `replace=True` to overwrite')
    parentdir=dst.parent
    if mkparent:
        parentdir.mkdir(parents=True, exist_ok=True)
    elif not parentdir.exists():
        raise FileNotFoundError(f'Parent directory {parentdir} does not exist')

    header = laspy.LasHeader(point_format=2, version='1.4')
    if pcd.crs:
        header.add_crs(pcd.crs)
    las = laspy.LasData(header)
    las.x = pcd.xyz[:,0]
    las.y = pcd.xyz[:,1]
    las.z = pcd.xyz[:,2]
    if pcd.rgb is not None:
        las.red = pcd.rgb[:,0]
        las.green = pcd.rgb[:,1]
        las.blue = pcd.rgb[:,2]
    if pcd.classification is not None:
        las.classification = pcd.classification
    las.write(str(dst))
    print('wrote', pcd.xyz.shape[0], 'points')

In [55]:
pcd=PointCloud(
    xyz=sample['coord'].numpy(),
    rgb=None,
    crs=pyproj.CRS.from_epsg(3857),
    classification=sample['segment'].numpy(),
)
write_las_file(pcd, '../data/sample.las', replace=True)

wrote 22100 points


In [63]:
np.unique(sample['segment'].numpy(),return_counts=True)[1]

array([  626,   423,    20,     9,   232, 10444,     3,   848,  3794,
        2767,  2934])

Goal: use a pretrained PointTransformerV3 model to predict the segmentation of a point cloud, and visualize.